## 타이타닉 데이터에 '검증' 적용하기

In [1]:
# 타이타닉 데이터 업로드
import pandas as pd

titanic = pd.read_csv("./titanic.csv")
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [2]:
# 라벨 인코더로 성별 컬럼 만들기
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(titanic['sex'])
titanic['gender'] = le.transform(titanic['sex'])
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,gender
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",0
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",1
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",1
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0


In [3]:
# 결측 데이터 걸러내기
titanic = titanic[titanic['age'].notnull()]
titanic = titanic[titanic['fare'].notnull()]
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1045 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1045 non-null   float64
 1   survived   1045 non-null   float64
 2   name       1045 non-null   object 
 3   sex        1045 non-null   object 
 4   age        1045 non-null   float64
 5   sibsp      1045 non-null   float64
 6   parch      1045 non-null   float64
 7   ticket     1045 non-null   object 
 8   fare       1045 non-null   float64
 9   cabin      272 non-null    object 
 10  embarked   1043 non-null   object 
 11  boat       417 non-null    object 
 12  body       119 non-null    float64
 13  home.dest  685 non-null    object 
 14  gender     1045 non-null   int64  
dtypes: float64(7), int64(1), object(7)
memory usage: 130.6+ KB


In [4]:
# 전체 라벨 데이터와 특성 데이터 추출
y = titanic['survived'].values
X = titanic[['pclass', 'age', 'sibsp', 'parch', 'fare', 'gender']].values

In [5]:
# Standard Scaler 적용
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()
SS.fit(X)

X_ss = SS.transform(X)

In [6]:
# Logistic Regresssion과 K Fold 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

log_reg_kf = LogisticRegression(random_state=13, solver='liblinear')
kfold = KFold(n_splits=5)
cv_accuracy = []

In [7]:
# 교차 검증 수행
from sklearn.metrics import accuracy_score

for train_idx, test_idx in kfold.split(X_ss):
  X_train, X_test = X_ss[train_idx], X_ss[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]

  log_reg_kf.fit(X_train, y_train)

  pred = log_reg_kf.predict(X_test)
  accuracy = accuracy_score(y_test, pred)

  cv_accuracy.append(accuracy)

In [8]:
# 각 fold마다의 성능
cv_accuracy

[0.7655502392344498,
 0.8660287081339713,
 0.8229665071770335,
 0.7464114832535885,
 0.7081339712918661]

In [9]:
# 교차 검증의 평균값
import numpy as np

np.mean(cv_accuracy)

0.7818181818181819

In [10]:
# label로 사용하는 생존율은 50%가 아님
# 그래서 각 fold마다 비율이 다르게 됨
for train_idx, test_idx in kfold.split(X_ss):
  X_train, X_test = X_ss[train_idx], X_ss[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]

  print(np.unique(y_test, return_counts=True))

(array([0., 1.]), array([ 74, 135]))
(array([0., 1.]), array([107, 102]))
(array([0., 1.]), array([125,  84]))
(array([0., 1.]), array([155,  54]))
(array([0., 1.]), array([157,  52]))


In [11]:
# 클래스의 불균형이 있을 때 fold 구성에 반영하기
# Stratified KFold 불러오기
from sklearn.model_selection import StratifiedKFold

log_reg_kf = LogisticRegression(random_state=13, solver='liblinear')
skfold = StratifiedKFold(n_splits=5)
cv_accuracy = []

In [12]:
# 생존/비생존 불균형 정도가 각 fold에 반영된 것 확인
for train_idx, test_idx in skfold.split(X_ss, y):
  X_train, X_test = X_ss[train_idx], X_ss[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]

  print(np.unique(y_test, return_counts=True))

(array([0., 1.]), array([123,  86]))
(array([0., 1.]), array([123,  86]))
(array([0., 1.]), array([124,  85]))
(array([0., 1.]), array([124,  85]))
(array([0., 1.]), array([124,  85]))


In [13]:
# Stratified KFold를 이용한 교차 검증
for train_idx, test_idx in kfold.split(X_ss, y):
  X_train, X_test = X_ss[train_idx], X_ss[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]

  log_reg_kf.fit(X_train, y_train)

  pred = log_reg_kf.predict(X_test)
  accuracy = accuracy_score(y_test, pred)

  cv_accuracy.append(accuracy)

In [14]:
# accuracy 확인
cv_accuracy

[0.7655502392344498,
 0.8660287081339713,
 0.8229665071770335,
 0.7464114832535885,
 0.7081339712918661]

In [15]:
# accuracy 평균 황긴
np.mean(cv_accuracy)

0.7818181818181819

In [16]:
# 반복문에서 수행하는 교차검증이 귀찮다면 : cross_val_score
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg_kf, X_ss, y, scoring='accuracy', cv=5)

In [17]:
scores

array([0.56937799, 0.86124402, 0.81339713, 0.73684211, 0.65550239])

In [18]:
np.mean(scores)

0.7272727272727273

## 타이타닉 데이터에 로지스틱 회귀 모델 '평가'하기